In [ ]:
import pandas as pd
import os
import pymysql
import configparser

# config open
config = configparser.ConfigParser()
config.read('C:\Users\SDA04\Desktop\final_project\esgpjt-2\config\config.ini')

conn = pymysql.connect(           
            # Local DB
            host = config['DB_TEST']['HOST'],
            port = 3306,
            user = config['DB_TEST']['USER'],
            password = config['DB_TEST']['PASSWD'],
            db = config['DB_TEST']['DB_NAME'],       
            charset = 'utf8mb4',
            cursorclass = pymysql.cursors.DictCursor # DB조회시 컬럼명을 동시에 보여줌
        )

cursor = conn.cursor()

# company_seq 를 찾아올 기업 테이블을 딕셔너리로 만들어 트래픽 줄이기
cursor.execute("SELECT company_name, company_seq FROM TB_COMPANY")
company_temp = cursor.fetchall()
company_dict = dict(company_temp)

# key_code 를 찾아올 키워드 테이블을 딕셔너리로 만들어 트래픽 줄이기
cursor.execute("SELECT key_word, key_code FROM TB_KEYWORD")
keyword_temp = cursor.fetchall()
keyword_dict = dict(keyword_temp)

# CSV 파일 경로
folder_path = r'C:\Users\SDA04\Desktop\yanghs81\proj_프로젝트\2_final_ESG\cleansing_folder'

# 기업별 뉴스 csv파일을 차례로 읽어 DB에 insert
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        print(file_name, '을 입력합니다.')

        file_path = os.path.join(folder_path, file_name)

        # CSV 파일을 읽어와서 DataFrame으로 변환
        df = pd.read_csv(file_path)

        # 데이터 삽입을 위한 SQL 쿼리 생성
        insert_query = """
            INSERT INTO tb_company_news (company_seq, key_code, news_press, news_title, news_article, write_date) 
            VALUES (%s, %s, %s, %s, %s, %s)
        """

        # 리스트를 만들어서 executemany에 넣을 데이터 준비
        data_to_insert = []
        for index, row in df.iterrows():
            data_to_insert.append((company_dict[row['company']], keyword_dict[row['keyword']], 'edaily', row['title'], row['article'], row['date']))

        # executemany로 한 번에 데이터 삽입
        cursor.executemany(insert_query, data_to_insert)

# 끝 처리        
print('뉴스 csv파일을 DB로 모두 입력완료 하였습니다.')
conn.commit()
conn.close()
